### Building a predictive model by using probability of past records

In [130]:
import csv
import pandas as pd
import numpy as np
import glob

In [131]:
path =r'/media/romit/249E6A809E6A49FE/my_work/core_project/fol' # use your path
allFiles = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
frame= pd.concat(list_)
len(list_)

frame['winner_name'].replace(' ', '_',regex=True,inplace=True)
frame['loser_name'].replace(' ', '_',regex=True,inplace=True)

In [132]:
x=frame['winner_name'].unique().tolist()
v=frame['loser_name'].unique().tolist()
res_f=list(set(x+v))

In [133]:
frame['winner']=frame[['winner_name']]
frame=frame[['winner_name','loser_name','surface','round','winner','w_SvGms','l_SvGms']].copy()


In [134]:
dfs=pd.read_csv('serve_analysis.csv')
dfr=pd.read_csv('return_analysis.csv')
dfrnd=pd.read_csv('round_abilities.csv')
dfsrf=pd.read_csv('surface_abilities.csv')

In [135]:
dfrnd['Player_name'].replace(' ', '_',regex=True,inplace=True)
dfsrf['Player_name'].replace(' ', '_',regex=True,inplace=True)

In [136]:
x1=dfs['Player_name'].tolist()
x2=dfr['Player_name'].tolist()
x3=dfrnd['Player_name'].tolist()
x4=dfsrf['Player_name'].tolist()

x=set(x1) & set(x2)& set(x3) & set(x4)
len(x)

94

In [137]:
def player_prof(dfs,dfr,dfrnd,dfsrf,res_f):
    player=[]
    cols=['Player_name','Serve','Return','Surface','Round']
    pp=pd.DataFrame(columns=cols)
    
    for z in range(len(res_f)):
        array=[]
        array.append(res_f[z])
        serve=(dfs[dfs['Player_name']==res_f[z]]['f(i)']).sum()
        array.append(serve)
        
        ret=(dfr[dfr['Player_name']==res_f[z]]['g(i)']).sum()
        array.append(ret)
        
        r1=(dfrnd[dfrnd['Player_name']==res_f[z]]['R128']).sum()
        array.append(r1)
        
        r2=(dfrnd[dfrnd['Player_name']==res_f[z]]['R64']).sum()
        array.append(r2)
        
        r3=(dfrnd[dfrnd['Player_name']==res_f[z]]['R32']).sum()
        array.append(r3)
        
        r4=(dfrnd[dfrnd['Player_name']==res_f[z]]['R16']).sum()
        array.append(r4)
        
        QF=(dfrnd[dfrnd['Player_name']==res_f[z]]['QF']).sum()
        array.append(QF)
        
        SF=(dfrnd[dfrnd['Player_name']==res_f[z]]['SF']).sum()
        array.append(SF)
        
        F=(dfrnd[dfrnd['Player_name']==res_f[z]]['F']).sum()
        array.append(F)
        
        grass=(dfsrf[dfsrf['Player_name']==res_f[z]]['W_Grass']).sum()
        array.append(grass)
        
        clay=(dfsrf[dfsrf['Player_name']==res_f[z]]['W_Clay']).sum()
        array.append(clay)
        
        carpet=(dfsrf[dfsrf['Player_name']==res_f[z]]['W_Carpet']).sum()
        array.append(carpet)
    
        hard=(dfsrf[dfsrf['Player_name']==res_f[z]]['W_Hard']).sum()
        array.append(hard)
        
        player.append(array)
        
    return player

In [138]:
t=player_prof(dfs,dfr,dfrnd,dfsrf,res_f)


In [139]:
columns=['Player_name','serve','return','r1','r2','r3','r4','QF','SF','F','Grass','Clay','Carpet','Hard']
play_prof=pd.DataFrame(t,columns=columns)

In [140]:
play_prof=play_prof.dropna()

In [141]:
x=play_prof['Player_name'].tolist()
frame['count']=-3

for z in range(len(frame[:10])):
    count=0
    if frame.iloc[z]['winner_name'] in x:
        count=count+1
    if frame.iloc[z]['loser_name'] in x:
        count=count+1
    frame.iloc[z]['count']=count    
        
        

/home/romit/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [142]:
def count(frame,play_prof):
    array=[]
    for z in range(len(frame)):
        count=0
        if frame.iloc[z]['winner_name'] in x:
            count=count+1
        if frame.iloc[z]['loser_name'] in x:
            count=count+1
        #print count
        array.append(count) 

    return array

In [147]:
t=count(frame,play_prof)
l=pd.Series(t)

frame['count']=l

In [148]:
frame_m=frame[frame['count']==2]
frame_m.dropna()
frame_m=frame_m.drop(['count'],axis=1)

## EVALUATION OF MODEL

### frame_m= DataSet


### play_prof=Player Profiles

In [149]:
import math

In [150]:
def model(play_prof,frame):
    a=[]
    f=float(play_prof['serve'].mean())
    g=float(play_prof['return'].mean())
    for z in range(len(frame)):
        x=frame.iloc[z]['winner_name']
        y=frame.iloc[z]['loser_name']
        
        x_sg=frame.iloc[z]['w_SvGms']
        y_sg=frame.iloc[z]['w_SvGms']
        
        srf=frame.iloc[z]['surface']
        rnd=frame.iloc[z]['round']
        
        
        
        winner=play_prof[play_prof['Player_name']==x]
        
        srv_w=winner['serve'].sum()
        ret_w=winner['return'].sum()
        
        if srf=='Grass':
            wp_s=winner['Grass'].sum()
        if srf=='Clay':
            wp_s=winner['Clay'].sum()
        if srf=='Hard':
            wp_s=winner['Hard'].sum()
        if srf=='Carpet':
            wp_s=winner['Carpet'].sum()
            
        if rnd=='R128':
            wp_r=winner['r1'].sum()
        if rnd=='R64':
             wp_r=winner['r2'].sum()
        if rnd=='R32':
             wp_r=winner['r3'].sum()
        if rnd=='R16':
            wp_r=winner['r4'].sum()
        if rnd=='QF':
            wp_r=winner['QF'].sum()
        if rnd=='SF':
             wp_r=winner['SF'].sum()
        if rnd=='F':
             wp_r=winner['F'].sum()
        
    
        
        looser=play_prof[play_prof['Player_name']==y]
        
        srv_l=looser['serve'].sum()
        ret_l=looser['return'].sum()
        
        if srf=='Grass':
            lp_s=looser['Grass'].sum()
        if srf=='Clay':
            lp_s=looser['Clay'].sum()
        if srf=='Hard':
            lp_s=looser['Hard'].sum()
        if srf=='Carpet':
            lp_s=looser['Carpet'].sum()
            
        if rnd=='R128':
            lp_r==looser['r1'].sum()
        if rnd=='R64':
            lp_r=looser['r2'].sum()
        if rnd=='R32':
            lp_r=looser['r3'].sum()
        if rnd=='R16':
            lp_r=looser['r4'].sum()
        if rnd=='QF':
            lp_r=looser['QF'].sum()
        if rnd=='SF':
            lp_r=looser['SF'].sum()
        if rnd=='F':
            lp_r=looser['F'].sum()
            
        
        f_xy=((srv_w-f)+(ret_w-g))
        f_yx=(srv_l-f)+(ret_l-g)
        
        '''ww=math.ceil(x_sg*(srv_w))
        ll=math.ceil(y_sg*(srv_l))
        
        
        t_x=(ww+(y_sg-ll))
        t_y=(ll+(x_sg-ww))'''
        
        
      
        
        if f_xy>=f_yx:
            a.append(y)
        else:
            a.append(x)
        
            
    return a
        

In [151]:
ser=model(play_prof,frame_m)

In [152]:
total=0
count=0
for z in range(len(frame_m)):
    if frame_m.iloc[z]['winner_name']==ser[z]:
        total=total+1
        count=count+1
       
    else:
        total=total+1
       
        
print count,total,float(count)/total
    

6798 8455 0.804021289178


## thus the accuracy of the model is 81%

##### the round and surface ability is yet to be incorporated into the model.